## TODOs
* [x] findIntersections
* [x] createReference
* [x] findCorrespondance
* [x] getIntersections `main`
* [ ] findSquareCenters `main`
* [ ] organize the whole structure

In [134]:
import numpy as np
from skimage import io
from skimage import transform

%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [135]:
def findHoughIntersections(horLines, verLines):
    horLinesCount = horLines.shape[1]
    verLinesCount = verLines.shape[1]

    # Contain all intersection points between the passed lines (horizontal, vertical)
    xIntersections = np.zeros((horLinesCount, verLinesCount))
    yIntersections = np.zeros((horLinesCount, verLinesCount))

    for i in range(horLinesCount):
        rho1 = horLines[1, i]
        theta1 = horLines[0, i]

        horLine = np.array([[np.cos(np.deg2rad(theta1))], [np.sin(np.deg2rad(theta1))], [-rho1]])

        for j in range(verLinesCount):
            rho2 = verLines[1, j]
            theta2 = verLines[0, j]

            verLine = np.array([[np.cos(np.deg2rad(theta2))], [np.sin(np.deg2rad(theta2))], [-rho2]])
            intersection = np.cross(horLine, verLine, axis=0) # get cross product of lines to get the intersection point
            intersection /= intersection[2, 0] # convert from 3D point to 2D point

            xIntersections[i, j] = intersection[0]
            yIntersections[i, j] = intersection[1]
    return (xIntersections, yIntersections)
    
    
# horLines = np.array([
#     [86, 86, 85, 84, 83, 82, 84, 82, 81, 80, 78, 77, 74, 76],
#     [1178, 1255, 1387, 1517, 1663, 1822, 1796, 2006, 2220, 2464, 2800, 3045, 3137, 3159]
# ])

# verLines =  np.array([
#     [2, 4, -2, -9, -15, -21, -26, -28, -30, -32, -34, -36, -38, -40, -41, -43, -42, -44, -46],
#     [112, 184, 199, 311, 407, 476, 545, 481, 632, 556, 691, 606, 720, 631, 789, 673, 866, 759, 659]
# ])

#############################################################################################################

# horLines = np.array([
#     [85, 89, 87, 86, 86, 86, 87, 86, 85, 45, 85, 45, 84, 84, 85, 80, 83, 83, 83, 83],
#     [910, 849, 911, 1333, 1408, 1564, 1648, 1743, 1954, 2401, 2153, 2512, 2391, 2641, 2724, 2833, 2822, 2939, 3235, 3386]
# ])

# verLines = np.array([
#     [3, 0, -3, -6, -45, -9, -46, -12, -14, -45, -17, -20, -22, -22],
#     [589, 743, 895, 1040, -530, 1177, -294, 1304, 1462, 66, 1577, 1671, 1701, 1790]
# ])

# xIntersections, yIntersections = findIntersections(horLines, verLines)
# print(xIntersections)
# print()
# print(yIntersections)




In [136]:
def createReferenceIntersections(imgSize):
    xPoints = np.array([list(range(9))] * 9) * (imgSize / 8) + 1
    yPoints = np.transpose(xPoints)
    
    return (xPoints, yPoints)

# xIntersectionsRef, yIntersectionsRef = createReferenceIntersections(319)
# print(xIntersectionsRef)
# print()
# print(yIntersectionsRef)


In [137]:
def transfrom(transformation, xPoints, yPoints):
    x = transformation[0, 0] * xPoints + transformation[1, 0] * yPoints + transformation[2, 0]
    y = transformation[0, 1] * xPoints + transformation[1, 1] * yPoints + transformation[2, 1]
    z = transformation[0, 2] * xPoints + transformation[1, 2] * yPoints + transformation[2, 2]

    return (x / z, y / z)


# moving = np.array([[5, 6], [20, 10], [20, 30], [4, 88]]) 
# fixed = np.array([[0, 0], [0, 50], [50, 50], [50, 0]])
# tform = transform.ProjectiveTransform()
# tform.estimate(fixed, moving)
# v1 = np.array([[6], [2], [89]])
# v2 = np.array([[7], [3], [56]])

# tform = tform.params.transpose()
# intersections = transfrom(tform, v1, v2)
# print(tform)
# print()
# print(intersections)



In [138]:
def geoTransformation(xIntersectionsRef, yIntersectionsRef, xIntersections, yIntersections):
    pCornersRef = np.array([
        [xIntersectionsRef[0, 0], yIntersectionsRef[0, 0]],
        [xIntersectionsRef[0, -1], yIntersectionsRef[0, -1]],
        [xIntersectionsRef[-1, -1], yIntersectionsRef[-1, -1]],
        [xIntersectionsRef[-1, 0], yIntersectionsRef[-1, 0]]
    ])


    M = 6      # Number of lines to search in each direction
    DMIN = 10   # radius for the intersection detection

    nMatchesBest = 0   # Number of matches of best candidate found so far
    avgErrBest = 1e10   # The average error of the best candidate
    corners = None

    
    N1 = xIntersections.shape[0] # horizontal lines count
    N2 = xIntersections.shape[1] # vertical lines count

    M1 = round(N1 * 0.6)
    M2 = round(N2 * 0.6)

    for i1a in range(0, min(M1, N1)):
        for i1b in range(N1 - 1, max(N1 - M1, i1a + 8) - 1, -1):
            for i2a in range(0, min(M2, N2)):
                for i2b in range(N2 - 1, max(N2 - M2, i2a + 8) - 1, -1):

                    if i1a == i1b or i2a == i2b:
                        continue
                    
                    # Get the four corners corresponding to the intersections
                    # of lines (1a,2a), (1a,2b), (1b,2b, and (1b,2a).

                    pCornersImg = np.zeros((4,2))
                    pCornersImg[0] = [xIntersections[i1a, i2a], yIntersections[i1a, i2a]]
                    pCornersImg[1] = [xIntersections[i1a, i2b], yIntersections[i1a, i2b]]
                    pCornersImg[2] = [xIntersections[i1b, i2b], yIntersections[i1b, i2b]]
                    pCornersImg[3] = [xIntersections[i1b, i2a], yIntersections[i1b, i2a]]


                    # Make sure that points are in clockwise order.
                    # If not, exchange points 2 and 4.
                    v12 = pCornersImg[1] - pCornersImg[0]
                    v13 = pCornersImg[2] - pCornersImg[0]
                    if v12[0] * v13[1] - v12[1] * v13[0] < 0:
                        pCornersImg[1], pCornersImg[3] = pCornersImg[3], pCornersImg[1]

                    # Let the points start from left top
                    sortcorners = pCornersImg
                    sortcorners = sortcorners[:, 0] ** 2 + sortcorners[:, 1] ** 2
                    indecies = [index for index, corner in enumerate(sortcorners) if corner == min(sortcorners)]
                    index = indecies[0]
                    pCornersImg = np.roll(pCornersImg, -index, axis=0)

                    # Fit a homography using those four points.

                    transformation = transform.ProjectiveTransform()
                    transformation.estimate(pCornersImg, pCornersRef)
                    
                    xIntersectionsFlattened = xIntersections.flatten(order='F')
                    yIntersectionsFlattened = yIntersections.flatten(order='F')
                    xIntersectionsRefFlattened = xIntersectionsRef.flatten(order='F')
                    yIntersectionsRefFlattened = yIntersectionsRef.flatten(order='F')


                    xIntersectionsFlattened.resize((len(xIntersectionsFlattened), 1))
                    yIntersectionsFlattened.resize((len(yIntersectionsFlattened), 1))

                    xIntersectionsRefFlattened.resize((len(xIntersectionsRefFlattened), 1))
                    yIntersectionsRefFlattened.resize((len(yIntersectionsRefFlattened), 1))

                    # tuple of xs and ys
                    pIntersections = transfrom(transformation.params.transpose(), xIntersectionsFlattened, yIntersectionsFlattened)
                    pIntersectionsRef = (xIntersectionsRefFlattened, yIntersectionsRefFlattened)
                    
                    pIntersections = np.array(list(zip(pIntersections[0].flatten(), pIntersections[1].flatten())))
                    pIntersectionsRef = np.array(list(zip(pIntersectionsRef[0].flatten(), pIntersectionsRef[1].flatten())))
                    # print(pIntersections)
                    # # For each predicted reference point, find the closest
                    # # detected image point.
                    dPts = 1e6 * np.ones((pIntersectionsRef.shape[0], 1))
                    for i in range(pIntersectionsRef.shape[0]):
                        x = pIntersectionsRef[i, 0]
                        y = pIntersectionsRef[i, 1]
                        d = np.sqrt((x - pIntersections[:, 0]) ** 2 + (y - pIntersections[:, 1]) ** 2)
                        dmin = np.min(d)
                        dPts[i] = dmin

                    # If the distance is less than DMIN, count it as a match.
                    nMatches = np.sum(dPts < DMIN)
                    
                    # Calculate the avg error of the matched points.
                    avgErr = np.mean(dPts[dPts < DMIN])
                    
                    # Keep the best combination found so far, in terms of
                    # the number of matches and the minimum error.
                    if nMatches < nMatchesBest:
                        continue
                    
                    if nMatches == nMatchesBest and avgErr > avgErrBest:
                        continue
                    
                    # # Got a better combination; save it.
                    avgErrBest = avgErr
                    nMatchesBest = nMatches
                    corners = pCornersImg
                    
    return corners, avgErrBest, nMatchesBest

# corners, avgErr, nMatches = geoTransformation(xIntersectionsRef, yIntersectionsRef, xIntersections, yIntersections)
# print(corners, avgErr, nMatches)


In [139]:
def calcIntersections(corners):
    dist1 = corners[1] - corners[0]
    dist2 = corners[2] - corners[0]

    if dist1[0] * dist2[1] - dist1[1] * dist2[0] < 0:
        corners[1], corners[3] = corners[3], corners[1]


    correctedCorners = corners
    correctedCorners = correctedCorners[:, 0] ** 2 + correctedCorners[:, 1] ** 2
    indecies = [index for index, corner in enumerate(correctedCorners) if corner == min(correctedCorners)]
    index = indecies[0]
    correctedCorners = np.roll(corners, -index, axis=0)
    
    xIntersectionsRef = np.array([list(range(9))] * 9) * 1 + 1
    yIntersectionsRef = np.transpose(xIntersectionsRef)

    cornersRef = np.array([
        [1, 1],
        [9, 1],
        [9, 9],
        [1, 9]
    ])


    transformation = transform.ProjectiveTransform()
    transformation.estimate(cornersRef, correctedCorners)

    xIntersectionsRefFlattened = xIntersectionsRef.flatten(order='F')
    yIntersectionsRefFlattened = yIntersectionsRef.flatten(order='F')
   
    intersections = np.array(list(zip(xIntersectionsRefFlattened, yIntersectionsRefFlattened, np.ones((len(yIntersectionsRefFlattened), 1)))))

    transformed = np.matmul(transformation.params, intersections.transpose())
    intersections = np.zeros((2, len(yIntersectionsRefFlattened)))
    intersections[0, :] = transformed[0, :] / transformed[2, :]
    intersections[1, :] = transformed[1, :] / transformed[2, :]

    return intersections


In [140]:
def writeCorners(corners, filePath):
    with open(filePath, 'w') as f:
        for point in corners:
            f.write(f'{point[0]},{point[1]}\n')

In [141]:
def writeIntersections(intersections, filePath): 
    with open(filePath, 'w') as f:
        for axis in intersections:
            for i in range(len(axis)):
                f.write(f'{axis[i]}' + ('' if i == len(axis) - 1 else ','))
            f.write('\n')

In [142]:
''' MAIN '''

# Lines generated from hough transform
lines1 = np.array([
    [85, 89, 87, 86, 86, 86, 87, 86, 85, 45, 85, 45, 84, 84, 85, 80, 83, 83, 83, 83],
    [910, 849, 911, 1333, 1408, 1564, 1648, 1743, 1954, 2401, 2153, 2512, 2391, 2641, 2724, 2833, 2822, 2939, 3235, 3386]
])


lines2 = np.array([
    [3, 0, -3, -6, -45, -9, -46, -12, -14, -45, -17, -20, -22, -22],
    [589, 743, 895, 1040, -530, 1177, -294, 1304, 1462, 66, 1577, 1671, 1701, 1790]
])

referenceImgSize = 319

xIntersections, yIntersections = findHoughIntersections(lines1, lines2)
xIntersectionsRef, yIntersectionsRef = createReferenceIntersections(referenceImgSize)
corners, err, matches = geoTransformation(xIntersectionsRef, yIntersectionsRef, xIntersections, yIntersections)
intersections = calcIntersections(corners)

# print(corners)
# print(intersections)

writeCorners(corners, '../CSVs/corners.csv')
writeIntersections(intersections, '../CSVs/intersections.csv')



C:\Users\hazem\AppData\Local\Temp\ipykernel_26232\4155591269.py:6: RuntimeWarning: divide by zero encountered in true_divide
  return (x / z, y / z)
C:\Users\hazem\AppData\Local\Temp\ipykernel_26232\4155591269.py:6: RuntimeWarning: invalid value encountered in true_divide
  return (x / z, y / z)
C:\Users\hazem\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\core\fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\hazem\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\hazem\AppData\Local\Temp\ipykernel_26232\416311370.py:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tup

In [143]:
# import math
# import numpy as np
# import matplotlib.pyplot as plt

# from skimage import io
# from skimage import transform

# text = io.imread('2.jpg')

# src = np.array([[0, 0], [0, 320], [320, 320], [320, 0]])
# dst = np.array([[1290, 1430], [300, 2360], [2000, 3200], [2550, 1880]])

# tform3 = transform.ProjectiveTransform()
# tform3.estimate(src, dst)
# warped = transform.warp(text, tform3, output_shape=(320, 320))

# fig, ax = plt.subplots(nrows=2, figsize=(8, 8))

# ax[0].imshow(text, cmap=plt.cm.gray)
# ax[0].plot(dst[:, 0], dst[:, 1], '.b')
# ax[1].imshow(warped, cmap=plt.cm.gray)

# for a in ax:
#     a.axis('off')

# plt.tight_layout()
# plt.show()

In [144]:
# a = np.array([[4], [-2], [1]])
# b = np.array([[1], [-1], [3]])
# c = np.cross(a, b, axis=0)
# print(c)

# arr = np.zeros((4, 2))
# arr[1] = [5, 6]
# print(arr)

# a = np.array([
#     [1, 8],
#     [7, 72],
#     [9, 3],
# ])
# a = a.flatten()
# # a.resize((len(a), 1))
# print(a)
# s = np.sqrt(a)
# print(s)

# print(np.mean(a[a <= 7]))

# x = np.array([5, 8, 7, 6])
# y = np.array([23, 55, 78, 45])

# print(np.array(list(zip(a.flatten(), a.flatten()))))
# print(np.array(list(a[:, 0])+list(a[:, 1])))
# a = a.flatten(order='F')
# a.resize((len(a), 1))
# print(a)
# print(a.flatten())
# print(1e10)

# shit1, shit2 = np.meshgrid(np.arange(9), np.arange(9))
# print(shit1)
# print(shit2)